# Dask Bags


Dask Bag implements operations like `map`, `filter`, `groupby` and aggregations on collections of Python objects. It does this in parallel and in small memory using Python iterators. It is similar to a parallel version of itertools or a Pythonic version of the PySpark RDD.

Dask Bags are often used to do simple preprocessing on log files, JSON records, or other user defined Python objects.

Full API documentation is available here: http://docs.dask.org/en/latest/bag-api.html

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 6.78 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34023,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 6.78 GiB
Comm: tcp://127.0.0.1:34313,Total threads: 1
Dashboard: http://127.0.0.1:38985/status,Memory: 1.70 GiB
Nanny: tcp://127.0.0.1:43757,


## Create Random Data

We create a random set of record data and store it to disk as many JSON files.  This will serve as our data for this notebook.

In [2]:
import dask
import json
import os

os.makedirs('data', exist_ok=True)              # Create data/ directory

b = dask.datasets.make_people()                 # Make records of people
b.map(json.dumps).to_textfiles('data/*.json')   # Encode as JSON, write to disk

['/home/runner/work/dask-examples/dask-examples/data/0.json',
 '/home/runner/work/dask-examples/dask-examples/data/1.json',
 '/home/runner/work/dask-examples/dask-examples/data/2.json',
 '/home/runner/work/dask-examples/dask-examples/data/3.json',
 '/home/runner/work/dask-examples/dask-examples/data/4.json',
 '/home/runner/work/dask-examples/dask-examples/data/5.json',
 '/home/runner/work/dask-examples/dask-examples/data/6.json',
 '/home/runner/work/dask-examples/dask-examples/data/7.json',
 '/home/runner/work/dask-examples/dask-examples/data/8.json',
 '/home/runner/work/dask-examples/dask-examples/data/9.json']

## Read JSON data

Now that we have some JSON data in a file lets take a look at it with Dask Bag and Python JSON module.

In [3]:
!head -n 2 data/0.json

{"age": 21, "name": ["Ellan", "Cleveland"], "occupation": "Practice Manager", "telephone": "+1-(636)-655-1684", "address": {"address": "179 Waithman Park", "city": "University Park"}, "credit-card": {"number": "4556 5225 4859 9839", "expiration-date": "04/22"}}
{"age": 37, "name": ["Nery", "Potts"], "occupation": "Gaming Board Inspector", "telephone": "432-175-5680", "address": {"address": "305 Evelyn Pike", "city": "Grandville"}, "credit-card": {"number": "5254 2607 5933 0881", "expiration-date": "04/25"}}


In [4]:
import dask.bag as db
import json

b = db.read_text('data/*.json').map(json.loads)
b

dask.bag<loads, npartitions=10>

In [5]:
b.take(2)

({'age': 21,
  'name': ['Ellan', 'Cleveland'],
  'occupation': 'Practice Manager',
  'telephone': '+1-(636)-655-1684',
  'address': {'address': '179 Waithman Park', 'city': 'University Park'},
  'credit-card': {'number': '4556 5225 4859 9839',
   'expiration-date': '04/22'}},
 {'age': 37,
  'name': ['Nery', 'Potts'],
  'occupation': 'Gaming Board Inspector',
  'telephone': '432-175-5680',
  'address': {'address': '305 Evelyn Pike', 'city': 'Grandville'},
  'credit-card': {'number': '5254 2607 5933 0881',
   'expiration-date': '04/25'}})

## Map, Filter, Aggregate

We can process this data by filtering out only certain records of interest, mapping functions over it to process our data, and aggregating those results to a total value.

In [6]:
b.filter(lambda record: record['age'] > 30).take(2)  # Select only people over 30

({'age': 37,
  'name': ['Nery', 'Potts'],
  'occupation': 'Gaming Board Inspector',
  'telephone': '432-175-5680',
  'address': {'address': '305 Evelyn Pike', 'city': 'Grandville'},
  'credit-card': {'number': '5254 2607 5933 0881',
   'expiration-date': '04/25'}},
 {'age': 56,
  'name': ['Earl', 'Wilder'],
  'occupation': 'Doctor',
  'telephone': '1-913-352-8064',
  'address': {'address': '618 Orange Esplanade', 'city': 'Valdosta'},
  'credit-card': {'number': '3792 105002 91297', 'expiration-date': '03/19'}})

In [7]:
b.map(lambda record: record['occupation']).take(2)  # Select the occupation field

('Practice Manager', 'Gaming Board Inspector')

In [8]:
b.count().compute()  # Count total number of records

10000

## Chain computations

It is common to do many of these steps in one pipeline, only calling `compute` or `take` at the end.

In [9]:
result = (b.filter(lambda record: record['age'] > 30)
           .map(lambda record: record['occupation'])
           .frequencies(sort=True)
           .topk(10, key=1))
result

dask.bag<topk-aggregate, npartitions=1>

As with all lazy Dask collections, we need to call `compute` to actually evaluate our result.  The `take` method used in earlier examples is also like `compute` and will also trigger computation.

In [10]:
result.compute()

[('Paint Consultant', 16),
 ('Piano Tuner', 15),
 ('Public Relations Of?cer', 14),
 ('Telesales Person', 14),
 ('Picture Editor', 14),
 ('Masseur', 14),
 ('Gaming Club Proprietor', 13),
 ('Revenue Officer', 13),
 ('Bakery Assistant', 13),
 ('Welder', 13)]

## Transform and Store

Sometimes we want to compute aggregations as above, but sometimes we want to store results to disk for future analyses.  For that we can use methods like `to_textfiles` and `json.dumps`, or we can convert to Dask Dataframes and use their storage systems, which we'll see more of in the next section.

In [11]:
(b.filter(lambda record: record['age'] > 30)  # Select records of interest
  .map(json.dumps)                            # Convert Python objects to text
  .to_textfiles('data/processed.*.json'))     # Write to local disk

['/home/runner/work/dask-examples/dask-examples/data/processed.0.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.1.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.2.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.3.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.4.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.5.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.6.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.7.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.8.json',
 '/home/runner/work/dask-examples/dask-examples/data/processed.9.json']

## Convert to Dask Dataframes

Dask Bags are good for reading in initial data, doing a bit of pre-processing, and then handing off to some other more efficient form like Dask Dataframes.  Dask Dataframes use Pandas internally, and so can be much faster on numeric data and also have more complex algorithms.  

However, Dask Dataframes also expect data that is organized as flat columns.  It does not support nested JSON data very well (Bag is better for this).

Here we make a function to flatten down our nested data structure, map that across our records, and then convert that to a Dask Dataframe.

In [12]:
b.take(1)

({'age': 21,
  'name': ['Ellan', 'Cleveland'],
  'occupation': 'Practice Manager',
  'telephone': '+1-(636)-655-1684',
  'address': {'address': '179 Waithman Park', 'city': 'University Park'},
  'credit-card': {'number': '4556 5225 4859 9839',
   'expiration-date': '04/22'}},)

In [13]:
def flatten(record):
    return {
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'credit-card-number': record['credit-card']['number'],
        'credit-card-expiration': record['credit-card']['expiration-date'],
        'name': ' '.join(record['name']),
        'street-address': record['address']['address'],
        'city': record['address']['city']   
    }

b.map(flatten).take(1)

({'age': 21,
  'occupation': 'Practice Manager',
  'telephone': '+1-(636)-655-1684',
  'credit-card-number': '4556 5225 4859 9839',
  'credit-card-expiration': '04/22',
  'name': 'Ellan Cleveland',
  'street-address': '179 Waithman Park',
  'city': 'University Park'},)

In [14]:
df = b.map(flatten).to_dataframe()
df.head()

,age,occupation,telephone,credit-card-number,credit-card-expiration,name,street-address,city
0,21,Practice Manager,+1-(636)-655-1684,4556 5225 4859 9839,04/22,Ellan Cleveland,179 Waithman Park,University Park
1,37,Gaming Board Inspector,432-175-5680,5254 2607 5933 0881,04/25,Nery Potts,305 Evelyn Pike,Grandville
2,26,Hotel Consultant,(169) 376-8972,4076 2362 1512 9751,08/23,Mel Nolan,1141 Marview Turnpike,Farmington
3,20,Cable Jointer,(038) 975-6980,3444 117591 41058,06/20,Rogelio Guerrero,388 Reddy Terrace,Holladay
4,56,Doctor,1-913-352-8064,3792 105002 91297,03/19,Earl Wilder,618 Orange Esplanade,Valdosta


We can now perform the same computation as before, but now using Pandas and Dask dataframe.

In [15]:
df[df.age > 30].occupation.value_counts().nlargest(10).compute()

Paint Consultant           16
Piano Tuner                15
Telesales Person           14
Masseur                    14
Public Relations Of?cer    14
Picture Editor             14
Safety Officer             13
Welder                     13
Gaming Club Proprietor     13
Riding Instructor          13
Name: occupation, dtype: int64

## Learn More

You may be interested in the following links:

-  [Dask Bag Documentation](https://docs.dask.org/en/latest/bag.html)
-  [API Documentation](http://docs.dask.org/en/latest/bag-api.html)
-  [dask tutorial](https://github.com/dask/dask-tutorial), notebook 02, for a more in-depth introduction.